## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [ ]:
import time
import chassisml
import numpy as np
import getpass
import json
from io import StringIO
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn_pmml_model.ensemble import PMMLForestClassifier

## Enter credentials
Dockerhub creds and Modzy API Key

In [ ]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

## Load and Test Model from PMML File

In [4]:
# Prepare data
iris = load_iris()
X = pd.DataFrame(iris.data)
X.columns = np.array(iris.feature_names)
y = pd.Series(np.array(iris.target_names)[iris.target])
y.name = "Class"
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.33, random_state=123)

# Create sample data for testing later
with open("data/sample_iris.csv", "w") as f:
    Xte[:10].to_csv(f, index=False)

In [5]:
# Load model
clf = PMMLForestClassifier(pmml="models/randomForest.pmml")
labels = clf.classes_.tolist()

# Test model
clf.predict(Xte)
clf.score(Xte, yte)

1.0

## Prepare pre and post-processing methods
Initialize anything here that should persist across inference runs

In [6]:
def preprocess_inputs(raw_input_bytes):
    # load data
    inputs = pd.read_csv(StringIO(str(raw_input_bytes, "utf-8")))
    return inputs

def postprocess_outputs(raw_predictions):
    # process output
    inference_result = {
        "result":[
            {
                "row": i+1,
                "classPredictions": [
                    {"class": labels[idx], "score": results[idx]}
                    for idx in np.argsort(results)[::-1]
                ]  
            } for i, results in enumerate(raw_predictions)
        ] 

    }    
    
    
    # format output
    structured_output = {
        "data": {
            "result": inference_result["result"],
            "explanation": None,
            "drift": None,
        }
    }
    
    return structured_output

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [9]:
def process(input_bytes):
    # load data
    inputs = preprocess_inputs(input_bytes)
    # make predictions
    output = clf.predict_proba(inputs)
    # process output
    structured_output = postprocess_outputs(output)
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [14]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com")

## Create and test Chassis model
* Requires `process_fn` defined above

In [15]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/sample_iris.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":[{"row":1,"classPredictions":[{"class":"versicolor","score":0.76},{"class":"virginica","score":0.24},{"class":"setosa","score":0.0}]},{"row":2,"classPredictions":[{"class":"virginica","score":0.995},{"class":"versicolor","score":0.005},{"class":"setosa","score":0.0}]},{"row":3,"classPredictions":[{"class":"virginica","score":1.0},{"class":"versicolor","score":0.0},{"class":"setosa","score":0.0}]},{"row":4,"classPredictions":[{"class":"versicolor","score":1.0},{"class":"virginica","score":0.0},{"class":"setosa","score":0.0}]},{"row":5,"classPredictions":[{"class":"setosa","score":1.0},{"class":"virginica","score":0.0},{"class":"versicolor","score":0.0}]},{"row":6,"classPredictions":[{"class":"virginica","score":0.82},{"class":"versicolor","score":0.18},{"class":"setosa","score":0.0}]},{"row":7,"classPredictions":[{"class":"versicolor","score":0.995},{"class":"virginica","score":0.005},{"class":"setosa","score":0.0}]},{"row":8,"classPredictions":[{"class":"setosa","sc

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [ ]:
MODEL_NAME = "PMML Random Forest Iris Classification"
start_time = time.time()
response = chassis_model.publish(
    model_name=MODEL_NAME,
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
)

In [37]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
end_time = time.time()
if final_status['status']['succeeded'] == 1:
    print("Job Completed in {} minutes.\n\nView your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(round((end_time-start_time)/60, 5), dockerhub_user, "-".join(MODEL_NAME.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))